<a href="https://colab.research.google.com/github/juanflorezVe/COMP9058_lab1/blob/master/r00184264_juanflorez_assg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# Copying vervatim the code from the assignment
import tensorflow as tf
print(tf.__version__)
experiment = tf.Variable([1,2,3])
print(tf.divide(tf.reduce_sum(experiment),experiment.shape[0]))

2.4.1
tf.Tensor(2.0, shape=(), dtype=float64)


In [95]:
import tensorflow as tf
import numpy as np
import math

from keras.utils import np_utils

fashion_mnist = tf.keras.datasets.fashion_mnist

# load the training and test data    
(tr_x, tr_y), (te_x, te_y) = fashion_mnist.load_data()

# reshape the feature data
tr_x = tr_x.reshape(tr_x.shape[0], 784)
te_x = te_x.reshape(te_x.shape[0], 784)

# noramlise feature data
tr_x = tr_x / 255.0
te_x = te_x / 255.0

print( "Shape of training features ", tr_x.shape)
print( "Shape of test features ", te_x.shape)


# one hot encode the training labels and get the transpose
tr_y = np_utils.to_categorical(tr_y,10)
tr_y = tr_y.T
print ("Shape of training labels ", tr_y.shape)

# one hot encode the test labels and get the transpose
te_y = np_utils.to_categorical(te_y,10)
te_y = te_y.T
print ("Shape of testing labels ", te_y.shape)

Shape of training features  (60000, 784)
Shape of test features  (10000, 784)
Shape of training labels  (10, 60000)
Shape of testing labels  (10, 10000)


In [96]:
#Let's insert 784 parameters to each neuron
# A = W.T * X + b
# layer_pass will take the x matrix, pass it to the l layer applying the w weights and adding the b bias
#NeuralNetworksW5 slide 11
# m = 60000
# n = 784
# p = 200
def relu_layer_pass(w, x, b):
  t = tf.matmul(w,x)
  l = tf.add(t,b)
  h = tf.nn.relu(l)
  return h

def softmax_layer_pass(w, x, b):
  t = tf.matmul(w,x)
  l = tf.add(t,b)
  l_exp = tf.math.exp(l)
  sums = tf.reduce_sum(l_exp,0)
  h = l_exp/sums

  return h

#TODO: X is the matrix with the training instances,
#but I need a function that given an Error, E, and a
# vector with inputs, gives me the d_lambda and db

def loss_function(E, x=None):
  db = tf.divide(tf.reduce_sum(E),E.shape[0])
  print("shape of x {}, shape of E transposed {}".
        format(x.shape, tf.transpose(E).shape))
  temp = tf.matmul(x, tf.transpose(E))
  d_lambda = tf.divide(temp, E.shape[0])
  return db, d_lambda

In [97]:
def forward_pass(w1, w2, b1, b2, x):
  #pass the first layer
  h1 = relu_layer_pass(w1,x,b1)
  #pass the second layer
  h2 = softmax_layer_pass(w2,h1,b2)
  return h2
  #print(H1.shape)
  #print(H2.shape)



In [98]:
# from lecture (VectorizedLogistiRegression, page 50) March 9th.... 
# We have a set of trainning of 60000 rows (instances)
# and 748 columns ( Features)
# Our X vector has 784 rows (as fetarues) and 60000 columns (instances)
X = tf.transpose(tr_x)
X = tf.cast(X, tf.float32)
print("X shape is {}".format( X.shape))
# Let's create a vector W1 with the number of weights (784) for each neuron in one row
# so, it means 200 rows and 784 columns ( Features) for layer 1
W1 = tf.Variable(tf.random.normal([200,784], mean=0.0, stddev=0.05))
#The bias for the first layer of 200 neurons will be a vecoter of 200 rows (1 per neuron)
# and 1 column
B1 = tf.Variable(tf.random.uniform(shape=[200,1],dtype=tf.float32))


print ("W1 shape is{}".format(W1.shape))
print ("B1 shape is{}".format(B1.shape))

X shape is (784, 60000)
W1 shape is(200, 784)
B1 shape is(200, 1)


In [99]:
#The second layer:
# W2 has 10 rows (as it has 10 possible classes)
W2 = tf.Variable(tf.random.uniform(shape=[10,200], dtype=tf.float32, minval=-0.1, maxval=0.1))
B2 = tf.Variable(tf.random.uniform(shape=[10,1], dtype=tf.float32))
print ("W2 shape is{}".format(W2.shape))
print ("B2 shape is{}".format(B2.shape))

W2 shape is(10, 200)
B2 shape is(10, 1)


Question 1, section a. Question1_1_1


In [100]:
learning_rate=0.01
iterations=1
adam_optimizer=tf.keras.optimizers.Adam()


In [101]:
# Rgression loop
for i in range(iterations):
  H = forward_pass(W1,W2,B1,B2,X)
  H1 = relu_layer_pass(W1, X, B1)
  H2 = softmax_layer_pass(W2, H1, B2)
  print("A column of H2 {}".format(H2[:,7489]))
  print("Sum of first column of H2 {}".format(tf.reduce_sum(H2[:,7489])))

  E = tf.subtract(H,tr_y)
  
  print("E shape is {}".format(E.shape))
  print("W1 shape is {}".format(W1.shape))
  #print("H shape is {}".format(H.shape))
  print("H1 shape is {}".format(H1.shape))
  print("H2 shape is {}".format(H2.shape))
  print("X shape is {}".format(X.shape))
  db2, dl2 = loss_function(E, H1)
  print("dl2 shape is {}".format(dl2.shape))
  B2 = tf.subtract(B2, tf.scalar_mul(learning_rate, db2))
  secondpart2 = tf.transpose(tf.scalar_mul(learning_rate, dl2)) #TODO WTF??? WHY?
  print("secondpart shape is {}".format(secondpart2.shape))
  W2 = tf.subtract(W2,secondpart2)
  db1, dl1 = loss_function(E, X)
  print("dl1 shape is {}".format(dl1.shape))
  B1 = tf.subtract(B1, tf.scalar_mul(learning_rate, db1))
  secondpart1 = tf.transpose(tf.scalar_mul(learning_rate, dl1)) #TODO WTF??? WHY?
  print("secondpart shape is {}".format(secondpart1.shape))
  W1 = tf.subtract(W1,secondpart1)
  print()
print("Done") 
    

A column of H2 [0.22727203 0.07542747 0.0402059  0.08087146 0.23783547 0.03462408
 0.07598529 0.04618661 0.04790181 0.13368995]
Sum of first column of H2 1.0000001192092896
E shape is (10, 60000)
W1 shape is (200, 784)
H1 shape is (200, 60000)
H2 shape is (10, 60000)
X shape is (784, 60000)
shape of x (200, 60000), shape of E transposed (60000, 10)
dl2 shape is (200, 10)
secondpart shape is (10, 200)
shape of x (784, 60000), shape of E transposed (60000, 10)
dl1 shape is (784, 10)
secondpart shape is (10, 784)


InvalidArgumentError: ignored

In [102]:
#Unit tests
xu = tf.Variable([[0.3,0.2,0.1],
                  [0.7,0.6,0.5],
                  [0.2,0.2,0.2]])
big = tf.Variable([[6.0,8.0,10.0]], shape = [1,3])
small = tf.Variable([5.0,7.0,9.0])
E = tf.math.subtract(big, small)
minus_E = tf.math.subtract(small, big)
print(loss_function(E, xu)) # should be xu/3
print(loss_function(minus_E, xu)) # should be -xu/3

shape of x (3, 3), shape of E transposed (3, 1)
(<tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.6],
       [1.8],
       [0.6]], dtype=float32)>)
shape of x (3, 3), shape of E transposed (3, 1)
(<tf.Tensor: shape=(), dtype=float32, numpy=-3.0>, <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.6],
       [-1.8],
       [-0.6]], dtype=float32)>)


In [103]:
scalar = tf.Variable(3.0)
vector = tf.Variable([3,6,9], dtype=tf.float32)
prod = tf.scalar_mul(scalar, vector)

In [104]:
prod

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 9., 18., 27.], dtype=float32)>

In [105]:
# implementing  slide 31 of NeuralNetworks5
A = tf.Variable([[6,-2, 3]], dtype = tf.float32)
t = tf.math.exp(A)
t

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4.0342880e+02, 1.3533528e-01, 2.0085537e+01]], dtype=float32)>

In [106]:
sum = tf.reduce_sum(t)
sum

<tf.Tensor: shape=(), dtype=float32, numpy=423.6497>

In [107]:
htest = t/sum
htest

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[9.5226979e-01, 3.1945092e-04, 4.7410719e-02]], dtype=float32)>

In [108]:
totalTest = tf.reduce_sum(htest)
totalTest

<tf.Tensor: shape=(), dtype=float32, numpy=0.99999994>

In [117]:
# Slide 36, calculate the loss function for softmax
t_expe = tf.Variable([[1.0], [0.0], [0.0]], shape=[3,1])
log_pr = -1*(tf.math.log(htest))
log_pr

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.04890689, 8.048907  , 3.048907  ]], dtype=float32)>

In [118]:
res = tf.matmul(log_pr,t_expe)
res

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.04890689]], dtype=float32)>

In [119]:
triv = tf.Variable(tf.exp(1.0), dtype=tf.float32)
proof = tf.math.log(triv)
proof

<tf.Tensor: shape=(), dtype=float32, numpy=0.99999994>